In [120]:
import numpy as np
import pandas as pd
import dill as pickle
from collections import defaultdict
import csv  # imports the csv module
import sys  # imports the sys module
import os
import timeit
import random
import copy

In [121]:
def validIP(address):  # Check if value is an ip address
    parts = address.split(".")
    if len(parts) != 4:
        return False
    for item in parts:
        if not 0 <= int(item) <= 255:
            return False
    return True

In [122]:
malIPS=[]
aa=open("maliciousips.txt")
for a in aa:
    a=a.rstrip()
    malIPS.append(a)
print(malIPS)

['192.168.2.112', '198.164.30.2', '192.168.2.113', '192.168.2.112', '147.32.84.180', '147.32.84.140', '10.0.2.15', '172.16.253.130', '172.16.253.240', '192.168.3.35', '172.29.0.116', '192.168.248.165', '131.202.243.84', '192.168.2.110', '192.168.1.103', '192.168.2.109', '147.32.84.170', '147.32.84.130', '192.168.106.141', '172.16.253.131', '74.78.117.238', '192.168.3.25', '172.29.0.109', '10.37.130.4', '192.168.5.122', '192.168.4.118', '192.168.4.120', '192.168.2.105', '147.32.84.150', '147.32.84.160', '192.168.106.131', '172.16.253.129', '158.65.110.24', '192.168.3.65', '172.16.253.132']


In [123]:
all_lens=[]
from collections import defaultdict 


In [124]:
mal_lengths=[]
beingn_lengths=[]
mal=0
nonmal=0

In [125]:
mal_flows = []
benign_flows=[]

In [111]:
def readLogs():
    n_flows = [10]
    cols = ['ts','uid','orig_h','id.orig_p','resp_h','id.resp_p','proto','service','duration','orig_bytes','resp_bytes','conn_state','local_orig','local_resp','missed_bytes','history','orig_pkts','orig_ip_bytes','resp_pkts','resp_ip_bytes','tunnel_parents']
    data = pd.read_csv('ICSX/testDset.log' , sep = '\t', skiprows=8, names = cols)  #prev : test_conn.log
    data.drop(data.tail(1).index,inplace=True) 
    newCol = data[['proto','service','conn_state']].apply(
             lambda x: '|'.join(map(str, x)), axis=1)
    data['state_proto_service'] = newCol
    unique_IPs = list(set(data['orig_h']))
    cleaned_unique_IPs = []
    for address in unique_IPs:
        if validIP(address):
            cleaned_unique_IPs.append(address)
    nonmal=0
    for ip in cleaned_unique_IPs:
        result = data[(data.resp_h ==ip) | (data.orig_h ==ip)]
        if ip in malIPS:
            print("Mal")
        else:
            #print(len(result))
            if (len(result))<100:
                    continue
            nonmal=nonmal+1
            n=1
            list_df = [result[i:i+n] for i in range(0,result.shape[0],1)]
            sigflow={}
            cc=0
            for item in list_df:
            #        #print(item)
                    #conn = list(item['conn_state'])
                    ts=str(list(item["ts"])[0])
                    sig = list(item['state_proto_service'])[0]
                    sigflow[ts]=sig
                    #print(sig)
                    #benign_flows[sig]=benign_flows[sig]+1
            sigflowordered = {k : sigflow[k] for k in sorted(sigflow)}
            benign_flows.append(list(sigflowordered.values()))
    print(nonmal)
    '''
    all_sigs=[]
    all_cons=[]    
    n=5
    list_df = [result[i:i+n] for i in range(0,result.shape[0],1)]
    print("-----")
    for item in list_df:
        conn = list(item['conn_state'])
        sig = list(item['state_proto_service'])
        print(sig)
    '''
    #print(dictmatrix_conn)
    #print(dictmatrix_sig)
    #print(all_cons)
    #print(all)

In [130]:
def readMultipleLogs():
    n_flows = [10]
    cols = ['ts','uid','orig_h','id.orig_p','resp_h','id.resp_p','proto','service','duration','orig_bytes','resp_bytes','conn_state','local_orig','local_resp','missed_bytes','history','orig_pkts','orig_ip_bytes','resp_pkts','resp_ip_bytes','tunnel_parents']
    xx=open("MalwareTrafficAnalysis/MALREC/all_malrec_files.txt")
    mal=0
    for x in xx:
        x=x.rstrip()
        data = pd.read_csv("MalwareTrafficAnalysis/MALREC/AllMalRecs/"+x , sep = '\t', skiprows=8, names = cols)
        data.drop(data.tail(1).index,inplace=True) 
        newCol = data[['proto','service','conn_state']].apply(
                 lambda x: '|'.join(map(str, x)), axis=1)
        data['state_proto_service'] = newCol
        unique_IPs = list(set(data['orig_h']))
        cleaned_unique_IPs = []
        for address in unique_IPs:
            if validIP(address):
                cleaned_unique_IPs.append(address)
        nonmal=0
        for ip in cleaned_unique_IPs:
            result = data[(data.resp_h ==ip) | (data.orig_h ==ip)]
            if 1:
                mal=mal+1
                if (len(result))<100:
                    continue
                #print(len(result))
                all_lens.append(len(result))
                n=1
                list_df = [result[i:i+n] for i in range(0,result.shape[0],1)]
                sigflow={}
                cc=0
                last_ts=""
                for item in list_df:
                    #print(item)
                #    conn = list(item['conn_state'])
                    ts=str(list(item["ts"])[0])
                    sig = list(item['state_proto_service'])[0]
                    sigflow[ts]=sig
                    #print(sig)
                    #mal_flows[sig]=mal_flows[sig]+1
                #Reorder sigflow 
                sigflowordered = {k : sigflow[k] for k in sorted(sigflow)}
                mal_flows.append(list(sigflowordered.values()))
        #print(nonmal)
    '''
    all_sigs=[]
    all_cons=[]    
    n=5
    list_df = [result[i:i+n] for i in range(0,result.shape[0],1)]
    print("-----")
    for item in list_df:
        conn = list(item['conn_state'])
        sig = list(item['state_proto_service'])
        print(sig)
    '''
    #print(dictmatrix_conn)
    #print(dictmatrix_sig)
    #print(all_cons)
    #print(all)

In [131]:
readMultipleLogs()
#print("Now Benign")
#readLogs()

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [106]:
print(benign_flows[0])

['udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'udp|-|S0', 'ud

In [93]:
#with open('filename','wb') as f: pickle.dump(arrayname, f)

#with open('filename','rb') as f: arrayname1 = pickle.load(f)

In [132]:
print(len(mal_flows))

160


In [133]:
'''
import pickle
with open('malr_sequences_v2','wb') as f:
    pickle.dump(mal_flows,f)
#with open('benign_sequences','wb') as f:
#    pickle.dump(benign_flows,f)
'''

In [156]:
mals_seqs=[]
with open('malrec_sequences_v2','rb') as f:
    mals_seqs.extend(pickle.load(f))
with open('mal_sequences_v2','rb') as f:
    mals_seqs.extend(pickle.load(f))

In [157]:
benign_seqs=[]
with open("benign_sequences_v2",'rb') as f:
    benign_seqs=pickle.load(f)

In [158]:
mals_calls=defaultdict(int)
for seq in mals_seqs:
    for item in seq:
        mals_calls[item]+=1

In [159]:
benign_calls=defaultdict(int)
for seq in benign_seqs:
    for item in seq:
        benign_calls[item]+=1

In [160]:
print(len(benign_calls.keys()))

36


In [161]:
all_calls=list(set(list(benign_calls.keys())+list(mals_calls.keys())))

In [162]:
print(len(all_calls))

65


In [165]:
call_encoding={}
for x in range(0,len(all_calls)):
    call_encoding[all_calls[x]]=x

In [12]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))

In [167]:
##Most awkward part : do the refining 

In [169]:
non_repeated_mals=[]
for seq in mals_seqs:
    last=""
    lxs=[]
    for item in seq:
        if item!=last:
            lxs.append(item)
            last=item
    non_repeated_mals.append(lxs)
print(len(non_repeated_mals))       

281


In [177]:
non_repeated_benigns=[]
for seq in benign_seqs:
    last=""
    lxs=[]
    for item in seq:
        if item!=last:
            lxs.append(item)
            last=item
    non_repeated_benigns.append(lxs)
print(len(non_repeated_benigns))       

1647


In [235]:
def f(a):
    length_a = len(a)
    limit = 200
    if length_a > limit:
      a = a[:limit]
    else:
      for i in range(0,limit - length_a):
        a.append(65)
    return a

In [236]:
max_clipping=200
encoded_benigns=[]
for long in non_repeated_benigns:
    if len(long)<20:
        continue
    lx=[]
    for item in long:
        lx.append(call_encoding[item])
    #Clip to 200 
    lx=lx[0:200] #truncate
    lx=f(lx) #clip
    encoded_benigns.append(lx)

In [237]:
encoded_mals=[]
for long in non_repeated_mals:
    if len(long)<20:
        continue
    lx=[]
    for item in long:
        lx.append(call_encoding[item])
    lx=lx[0:200]
    lx=f(lx)
    encoded_mals.append(lx)

In [238]:
X=[]
Y=[]
for item in encoded_benigns:
    X.append(item)
    Y.append(0)
for item in encoded_mals:
    X.append(item)
    Y.append(1)

In [239]:
with open('export_X','wb') as f:
    pickle.dump(X,f)
with open('export_Y','wb') as f:
    pickle.dump(Y,f)

In [216]:
from sklearn.model_selection import train_test_split

In [222]:
X_train, X_test, Y_train, Y_test = train_test_split(
...    X, Y, test_size=0.20, random_state=1000)

In [227]:
import numpy as np
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
Y_train=np.asarray(Y_train)
Y_test=np.asarray(Y_test)

In [229]:
import keras,tensorflow

In [234]:
print(call_encoding)

{'tcp|-|REJ': 0, 'tcp|-|SH': 1, 'udp|-|SF': 2, 'tcp|smtp|RSTR': 3, 'udp|teredo|SF': 4, 'tcp|-|S3': 5, 'udp|dhcp|SF': 6, 'tcp|smtp|S2': 7, 'tcp|smb,gssapi|RSTR': 8, 'udp|ntp|SF': 9, 'tcp|gssapi|OTH': 10, 'tcp|http|S2': 11, 'tcp|smb|RSTR': 12, 'udp|dns|SHR': 13, 'tcp|-|SF': 14, 'tcp|ssl|RSTO': 15, 'tcp|http|RSTO': 16, 'tcp|dce_rpc,ntlm|OTH': 17, 'tcp|smtp|S3': 18, 'tcp|krb_tcp|RSTR': 19, 'tcp|http|SF': 20, 'udp|dhcp|SHR': 21, 'icmp|-|OTH': 22, 'udp|-|SHR': 23, 'tcp|dns|SF': 24, 'tcp|dce_rpc|SHR': 25, 'udp|dhcp|S0': 26, 'tcp|dce_rpc,smb,ntlm,gssapi|SF': 27, 'tcp|smtp|SF': 28, 'tcp|smtp,ssl|SF': 29, 'tcp|ssl|SF': 30, 'tcp|-|RSTO': 31, 'tcp|http|S3': 32, 'tcp|-|SHR': 33, 'tcp|ssl|S1': 34, 'tcp|smtp|S1': 35, 'tcp|ssl|RSTR': 36, 'tcp|dns|RSTR': 37, 'tcp|smtp,ssl|RSTO': 38, 'tcp|ssl|S2': 39, 'udp|ntp|S0': 40, 'tcp|http|S1': 41, 'tcp|-|S1': 42, 'tcp|dns|OTH': 43, 'tcp|irc|SH': 44, 'tcp|irc|OTH': 45, 'tcp|gssapi,smb,ntlm|SF': 46, 'tcp|-|RSTR': 47, 'tcp|-|S2': 48, 'tcp|gssapi,ntlm|OTH': 49, 'tcp|